In [1]:
import yfinance as yf
import datetime
import ta
import datetime
from ta import momentum, trend

symbol = "^NSEI"
ticker = yf.Ticker(symbol)
end_date = datetime.date.today()
df = ticker.history(period="10d", interval="5m")
df['Date'] = df.index.date
df['Time'] = df.index.time
df

,Open,High,Low,Close,Volume,Dividends,Stock Splits,Date,Time
Datetime,,,,,,,,,
2023-10-10 09:15:00+05:30,19565.599609,19606.949219,19565.599609,19599.900391,0,0.0,0.0,2023-10-10,09:15:00
2023-10-10 09:20:00+05:30,19600.849609,19602.449219,19591.849609,19597.300781,0,0.0,0.0,2023-10-10,09:20:00
2023-10-10 09:25:00+05:30,19597.500000,19598.650391,19585.199219,19591.650391,0,0.0,0.0,2023-10-10,09:25:00
2023-10-10 09:30:00+05:30,19591.400391,19594.800781,19583.500000,19592.599609,0,0.0,0.0,2023-10-10,09:30:00
2023-10-10 09:35:00+05:30,19592.150391,19600.500000,19587.599609,19600.199219,0,0.0,0.0,2023-10-10,09:35:00
...,...,...,...,...,...,...,...,...,...
2023-10-23 15:05:00+05:30,19316.199219,19320.199219,19284.099609,19286.050781,0,0.0,0.0,2023-10-23,15:05:00
2023-10-23 15:10:00+05:30,19286.650391,19289.199219,19269.550781,19272.699219,0,0.0,0.0,2023-10-23,15:10:00
2023-10-23 15:15:00+05:30,19273.550781,19282.849609,19271.550781,19281.699219,0,0.0,0.0,2023-10-23,15:15:00


In [2]:
from ta import momentum,trend
df['50EMA'] = trend.EMAIndicator(df['Close'], window=50).ema_indicator()
df['10EMA'] = trend.EMAIndicator(df['Close'], window=10).ema_indicator()

In [3]:
def Find_Up(df):
    data = df[0]<df[1]
    if data==True:
        return "Green"
    else:
        return "Red"
df['Canndel']= df[['Open','Close']].apply(Find_Up,axis=1)

In [4]:
df['SMA20'] = df['Close'].rolling(window=20).mean()
df['SMA40'] = df['Close'].rolling(window=40).mean()
df['Trend'] = 'Sideways'
df.loc[df['Close'] > df['SMA20'], 'Trend'] = 'Uptrend'
df.loc[df['Close'] < df['SMA20'], 'Trend'] = 'Downtrend'
del df['SMA20']
del df['SMA40']
del df['Volume']
del df['Dividends']
del df['Stock Splits']

In [5]:
# df.sort_values(by="Datetime", ascending=False, inplace=True)
# df.head(5)

In [6]:
HOLDING = False
inisal_amount = 30000

def find_10EMA(row):
    global HOLDING
    global inisal_amount
    lot_size = 50
    current_price = row['Close'] * lot_size
    
    if row['10EMA'] > row['Close']:
        if row['Canndel'] == "Green":
            if not HOLDING:
                print(f"PE ENTRY ---------------[{row['Date']} [{row['Time']}]]---------")
                inisal_amount = inisal_amount - current_price
                HOLDING = True
    elif row['10EMA'] < row['Close']:
        if row['Canndel'] == "Green":
            if HOLDING:
                print(f"PE EXIT ---------------[{row['Date']} [{row['Time']}]]---------")
                inisal_amount = inisal_amount + current_price
                HOLDING = False

    else:
        print("SIDEWAY")



In [7]:
df.apply(find_10EMA, axis=1)
if HOLDING == True :
    print("HOLDING")
    last_price_today = round(df.iloc[1]['Close'],2)*50
    print("MY AMOUNT : 30000 ")
    print("TOTAL RETURNS  : ",last_price_today + inisal_amount)
    print("MY P&L : ",last_price_today + inisal_amount-30000)
    percentage = (100 * (last_price_today + inisal_amount-30000))/30000 
    print(f"MY P& %: {percentage}%")
    
else:
    print("MY AMOUNT : 30000 ")
    print("TOTAL RETURNS  : ",inisal_amount)
    print("MY P&L : ",inisal_amount-30000)
    percentage = (100 * (inisal_amount-30000))/30000 
    print(f"MY P& %: {percentage}%")

SIDEWAY
SIDEWAY
SIDEWAY
SIDEWAY
SIDEWAY
SIDEWAY
SIDEWAY
SIDEWAY
SIDEWAY
PE ENTRY ---------------[2023-10-10 [15:15:00]]---------
PE EXIT ---------------[2023-10-11 [09:15:00]]---------
PE ENTRY ---------------[2023-10-11 [11:15:00]]---------
PE EXIT ---------------[2023-10-11 [11:25:00]]---------
PE ENTRY ---------------[2023-10-11 [12:20:00]]---------
PE EXIT ---------------[2023-10-11 [13:00:00]]---------
PE ENTRY ---------------[2023-10-11 [14:00:00]]---------
PE EXIT ---------------[2023-10-11 [14:10:00]]---------
PE ENTRY ---------------[2023-10-11 [15:20:00]]---------
PE EXIT ---------------[2023-10-12 [10:40:00]]---------
PE ENTRY ---------------[2023-10-12 [12:20:00]]---------
PE EXIT ---------------[2023-10-12 [12:45:00]]---------
PE ENTRY ---------------[2023-10-12 [13:00:00]]---------
PE EXIT ---------------[2023-10-12 [13:20:00]]---------
PE ENTRY ---------------[2023-10-12 [14:10:00]]---------
PE EXIT ---------------[2023-10-12 [14:30:00]]---------
PE ENTRY ---------------